In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import copy
from utils import *
from experiment import *
from definitions import *

In [2]:
min_models = 2 #no need to change this from 2
exp = 2 #no need to change this from 2

select_models = False

Defining test families and check which families should be delete (if any)

In [3]:
benchs_names_list = [['MMLU','ARC','HellaSwag','Winogrande','TruthfulQA','GSM8K'],
                     ['IFEval','BBH','MATH Lvl 5','GPQA','MUSR','MMLU-PRO'],
                     ['MMLU','ARC','HellaSwag','Winogrande','TruthfulQA','GSM8K','IFEval','BBH','MATH Lvl 5','GPQA','MUSR','MMLU-PRO']] 

n_train_models = min_models-1
benchs_names = benchs_names_list[exp]

if n_train_models==1:
    test_families_list = [[['bloom'],
                           ['codegen-nl'],
                           ['codellama'],
                           ['deepseek-coder-base'],
                           ['pythia','dolly-v2'],
                           ['falcon'],
                           ['gemma', 'gemma-it','sauerkrautlm-gemma'],
                           ['gpt-j-neo-neox'], 
                           ['internlm2'],
                           ['meta-llama-3', 'meta-llama-3-instruct'],
                           ['mpt', 'mpt-chat','mpt-instruct'],
                           ['olmo'],
                           ['opt'],
                           ['qwen2'],
                           ['rwkv-4-pile'],
                           #['rwkv-raven'],
                           ['starcoder2'],
                           ['stablelm-base-alpha'],
                           ['xglm'],
                           ['yi-1.5', 'yi-1.5-chat','dolphin-2.9.1-yi-1.5']],
                          [['bloom'],
                           ['pythia','dolly-v2'],
                           ['falcon','falcon-instruct'],
                           ['gemma-2', 'gemma-2-it'],
                           ['gpt-j-neo-neox'], 
                           ['meta-llama-3', 'meta-llama-3-instruct','llama-3-sauerkrautlm-instruct'],
                           ['olmo'],
                           ['opt'],
                           ['qwen2','qwen2-instruct','dolphin-2.9.2-qwen2'],
                           ['starcoder2'],
                           ['smollm', 'smollm-instruct'],
                           ['yi-1.5', 'yi-1.5-chat','dolphin-2.9.1-yi-1.5']],
                          [['bloom'],
                           ['pythia','dolly-v2'],
                           ['falcon'],
                           ['gemma', 'gemma-it', 'sauerkrautlm-gemma'],
                           ['gpt-j-neo-neox'], 
                           ['meta-llama-3', 'meta-llama-3-instruct'],
                           ['olmo'],
                           ['opt'],
                           ['qwen2'],
                           ['starcoder2'],
                           ['yi-1.5', 'yi-1.5-chat','dolphin-2.9.1-yi-1.5']]]
else:
    test_families_list = [[['bloom'],
                             ['codellama'],
                             ['deepseek-coder-base'], 
                             ['falcon'],
                             ['gpt-j-neo-neox'], 
                             ['llama-2', 'llama-2-chat'],
                             ['open_llama_'], 
                             ['opt'], 
                             ['pythia'], 
                             ['qwen1.5', 'qwen1.5-chat'],
                             ['qwen2'],
                             ['rwkv-4-pile'],
                             #['rwkv-raven'],
                             ['starcoder2'],
                             ['xglm'],
                             ['yi-1.5', 'yi-1.5-chat']],
                          [['bloom'],
                             ['llama-2', 'llama-2-chat'],
                             ['orca_mini_v3_'],
                             ['pythia','dolly-v2'],
                             ['qwen1.5', 'qwen1.5-chat'],
                             ['qwen2','qwen2-instruct'],
                             ['smollm', 'smollm-instruct'], 
                             ['starcoder2'],
                             ['yi-1.5', 'yi-1.5-chat']],
                          [['bloom'],
                             ['llama-2', 'llama-2-chat'], 
                             ['pythia'], 
                             ['qwen1.5','qwen1.5-chat'],
                             ['qwen2'], 
                             ['starcoder2'], 
                             ['yi-1.5','yi-1.5-chat']]]

if select_models:
    thresh_MMLU = .35
    thresh_MMLU_PRO = .15
    data = pd.read_csv('data/data_v2.csv')
    data = data.sort_values(by=['Family','#Params (B)'])
    biggest_model_data = data.drop_duplicates(subset=['Family'], keep='last')
    families_to_delete = np.unique(biggest_model_data.loc[(biggest_model_data.loc[:,'MMLU']<thresh_MMLU) | (biggest_model_data.loc[:,'MMLU-PRO']<thresh_MMLU_PRO)].Family).tolist()
    families_to_delete = np.unique(data.loc[[f in families_to_delete for f in data.Family]].Family2).tolist()
    test_families_list = [[x for x in y if not np.sum([z in families_to_delete for z in x])>0] for y in test_families_list]
    print('families_to_delete:',families_to_delete)

In [4]:
def get_results(test_families, benchs_names):
    data = pd.read_csv('data/data_v2.csv')
    if select_models:
        data = data.loc[[f not in families_to_delete for f in np.array(data['Family2'])]]
        
    data['Family'] = data['Family2']
    data, unique_families, avail_families = prep_data(data, benchs_names, min_models)

    X_train, X2_train, F_train, D_train, Y_train, X_test, X2_test, F_test, D_test, Y_test, Instruct_test = prep_data2(data, test_families, benchs_names, n_train_models=n_train_models)
    Inter_train = np.ones((X_train.shape[0],1))
    Inter_test = np.ones((X_test.shape[0],1))
    
    Cs = []
    for s in benchs_names:
        Cs.append(lower_bounds[s])
    Cs = np.array(Cs).astype(float)[None,:]

    if n_train_models==2:
        F_train = F_train*D_train
        F_test = F_test*D_test

    return run_exp2(X_train, D_train, Y_train, X_test, D_test, Y_test, Cs), Instruct_test, test_families

In [ ]:
errors = Parallel(n_jobs=-1, verbose=True)(delayed(get_results)(test_families, benchs_names) for test_families in test_families_list[exp])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.


In [ ]:
np.save(f'results/errors_exp2-{exp}_n-train-models-{n_train_models}_select-models-{select_models}.npy', {'out':errors})